<a target="_blank" href="https://colab.research.google.com/github/alexmelekhin/iprofihack2023_place_recognition/blob/dev/baseline_demo.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Установка в Google Colab

Ячейки ниже рекомендуется использовать для установки зависимостей в Google Colab.

1. Убедитесь, что вы подключены к окружению с GPU

In [ ]:
!nvidia-smi

2. По умолчанию установлена версия torch 2.0, нам нужно откатиться до 1.13.1 (код не тестировался на версии 2.0 и может вести себя непредсказуемо)

In [ ]:
!pip install torch==1.13.1 torchvision==0.14.1

In [ ]:
import torch
print(f"torch version: {torch.__version__}")
print(f"Is CUDA available in torch?: {torch.cuda.is_available()}")

3. Установка необходимых библиотек для сборки MinkowskiEngine

In [ ]:
!pip install ninja

In [ ]:
!sudo apt-get install libopenblas-dev

4. Сборка и установка MinkowskiEngine из исходников (занимает много времени)

In [ ]:
!pip install -U git+https://github.com/NVIDIA/MinkowskiEngine -v --no-deps \
                          --install-option="--force_cuda" \
                          --install-option="--blas=openblas"

5. Проверка, что все работает

In [ ]:
import torch
print(f"Is CUDA available in torch?: {torch.cuda.is_available()}")
import MinkowskiEngine as ME
print(f"Is CUDA available in MinkowskiEngine?: {ME.is_cuda_available()}")
ME.print_diagnostics()

6. Финальный шаг - установка библиотеки [opr](https://github.com/alexmelekhin/open_place_recognition), код из которой будет использоваться в бейзлайне

In [ ]:
!git clone https://github.com/alexmelekhin/open_place_recognition
%cd open_place_recognition
!pip install -e .  # флаг -e необходим для возможности редактировать код уже установленной библиотеки
%cd ..

## Загрузка датасета в Google Colab

Пример кода для загрузки датасета.

Вы можете воспользоваться утилитой gdown, которая по умолчанию доступна в Colab. Допустим, https://drive.google.com/file/d/1EdOTVgBJxsNUMecne7Fs4obJdJnDuJ18/view?usp=share_link - ссылка на файл. Чтобы скачать его, нам нужно передать в gdown в качестве аргумента его id - для данного примера это `1EdOTVgBJxsNUMecne7Fs4obJdJnDuJ18` (часть ссылки между `file/d/` и `/view`).

In [ ]:
!gdown 1EdOTVgBJxsNUMecne7Fs4obJdJnDuJ18

Вы можете сверить хэш-сумму файла:

In [ ]:
!sha256sum public.zip

И распаковать архив:

In [ ]:
!unzip -q public.zip

## Базовое решение

Загрузите веса MinkLoc++, предобученного на датасете Oxford RobotCar по ссылке: https://drive.google.com/file/d/1zlfdX217Nh3_QL5r0XAHUjDFjIPxUmMg/view?usp=share_link

In [ ]:
# если вы в colab'е:
# !gdown 1zlfdX217Nh3_QL5r0XAHUjDFjIPxUmMg

In [1]:
from opr.models import minkloc_multimodal

model = minkloc_multimodal(weights="baseline_minkloc_multimodal.pth")

/home/docker_hackathon/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pathlib import Path

EPOCHS = 10

IMAGE_LR = 0.0001
CLOUD_LR = 0.001
FUSION_LR = 0.001
WEIGHT_DECAY = 0.0001

SCHEDULER_GAMMA = 0.1
SCHEDULER_STEPS = [5]

DEVICE = "cuda"
BATCH_EXPANSION_TH = None
CHECKPOINTS_DIR = Path("checkpoints")


Для инициализации функции лосса предлагается воспользоваться средствами библиотеки [Hydra](https://hydra.cc/docs/intro/).

Примеры готовых конфиг-файлов есть в директории "configs" [репозитория opr](https://github.com/alexmelekhin/open_place_recognition). Обратите внимание, что в конфигурации датасета необходимо указать путь к его директории.

In [4]:
from torch.optim import Adam
from torch.optim.lr_scheduler import MultiStepLR

from hydra.utils import instantiate
from omegaconf import OmegaConf

from opr.datasets.dataloader_factory import make_dataloaders

LOSS_CFG_PATH = ...
DATASET_CFG_PATH = ...

loss_cfg = OmegaConf.load(LOSS_CFG_PATH)
loss_fn = instantiate(loss_cfg)

dataset_cfg = OmegaConf.load(DATASET_CFG_PATH)

dataset_cfg.dataset.dataset_root = ...  # change path

dataloaders = make_dataloaders(
    dataset_cfg=dataset_cfg.dataset,
    batch_sampler_cfg=dataset_cfg.sampler,
    num_workers=dataset_cfg.num_workers,
)

params_list = []
if model.image_module is not None and IMAGE_LR is not None:
    params_list.append({"params": model.image_module.parameters(), "lr": IMAGE_LR})
if model.cloud_module is not None and CLOUD_LR is not None:
    params_list.append({"params": model.cloud_module.parameters(), "lr": CLOUD_LR})
if model.fusion_module is not None and FUSION_LR is not None:
    params_list.append({"params": model.fusion_module.parameters(), "lr": FUSION_LR})
optimizer = Adam(params_list, weight_decay=WEIGHT_DECAY)
scheduler = MultiStepLR(optimizer, milestones=SCHEDULER_STEPS, gamma=SCHEDULER_GAMMA)

if not CHECKPOINTS_DIR.exists():
    CHECKPOINTS_DIR.mkdir(parents=True)

model = model.to(DEVICE)

In [5]:
from opr.testing import test


recall_at_n, recall_at_one_percent, mean_top1_distance = test(
    model=model,
    descriptor_key="fusion",
    dataloader=dataloaders["test"],
    dist_thresh=5.0,
    device=DEVICE,
)


Calculating metrics: 100%|██████████| 6/6 [00:00<00:00,  6.70it/s]

Mean Recall@N:
[0.35832433 0.46075096 0.52536504 0.5618218  0.59198689 0.61640536
 0.63445118 0.65197783 0.67150134 0.68306623 0.69275496 0.70492458
 0.71186098 0.71967074 0.72547996 0.73210702 0.73955678 0.74756834
 0.75144274 0.75862737 0.76363231 0.77028684 0.77800951 0.78381246
 0.78966836]
Mean Recall@1% = 0.6164053595498168
Mean top-1 distance = 1.0901976154417208
[0.35832433 0.46075096 0.52536504 0.5618218  0.59198689 0.61640536
 0.63445118 0.65197783 0.67150134 0.68306623 0.69275496 0.70492458
 0.71186098 0.71967074 0.72547996 0.73210702 0.73955678 0.74756834
 0.75144274 0.75862737 0.76363231 0.77028684 0.77800951 0.78381246
 0.78966836]


In [6]:
print(recall_at_n)

[0.35832433 0.46075096 0.52536504 0.5618218  0.59198689 0.61640536
 0.63445118 0.65197783 0.67150134 0.68306623 0.69275496 0.70492458
 0.71186098 0.71967074 0.72547996 0.73210702 0.73955678 0.74756834
 0.75144274 0.75862737 0.76363231 0.77028684 0.77800951 0.78381246
 0.78966836]


### Цикл обучения

In [ ]:
import torch
from opr.training import epoch_loop


best_recall_at_1 = 0.0

for epoch in range(EPOCHS):
    print(f"\n\n=====> Epoch {epoch+1}:")
    train_batch_size = dataloaders["train"].batch_sampler.batch_size

    print("\n=> Training:\n")

    train_stats, train_rate_non_zero = epoch_loop(
        dataloader=dataloaders["train"],
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        scheduler=scheduler,
        phase="train",
        device=DEVICE,
    )

    print(f"\ntrain_rate_non_zero = {train_rate_non_zero}")

    if BATCH_EXPANSION_TH is not None:
        if BATCH_EXPANSION_TH == 1.0:
            print("Batch expansion rate is set to every epoch. Increasing batch size.")
            dataloaders["train"].batch_sampler.expand_batch()
        elif train_rate_non_zero is None:
            print(
                "\nWARNING: 'BATCH_EXPANSION_TH' was set, but 'train_rate_non_zero' is None. ",
                "The batch size was not expanded.",
            )
        elif train_rate_non_zero < BATCH_EXPANSION_TH:
            print(
                "Average non-zero triplet ratio is less than threshold: ",
                f"{train_rate_non_zero} < {BATCH_EXPANSION_TH}",
            )
            dataloaders["train"].batch_sampler.expand_batch()

    print("\n=> Validating:\n")

    val_stats, val_rate_non_zero = epoch_loop(
        dataloader=dataloaders["val"],
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        phase="val",
        device=DEVICE,
    )

    print(f"\nval_rate_non_zero = {val_rate_non_zero}")

    print("\n=> Testing:\n")

    recall_at_n, recall_at_one_percent, mean_top1_distance = test(
        model=model,
        descriptor_key="fusion",
        dataloader=dataloaders["test"],
        dist_thresh=5.0,
        device=DEVICE,
    )

    stats_dict = {}
    stats_dict["test"] = {
        "mean_top1_distance": mean_top1_distance,
        "recall_at_1%": recall_at_one_percent,
        "recall_at_1": recall_at_n[0],
        "recall_at_3": recall_at_n[2],
        "recall_at_5": recall_at_n[4],
        "recall_at_10": recall_at_n[9],
    }
    stats_dict["train"] = train_stats
    stats_dict["train"]["batch_size"] = train_batch_size

    # saving checkpoints
    checkpoint_dict = {
        "epoch": epoch + 1,
        "stats_dict": stats_dict,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
    }
    torch.save(checkpoint_dict, CHECKPOINTS_DIR / f"epoch_{epoch+1}.pth")
    if recall_at_n[0] > best_recall_at_1:
        print("Recall@1 improved!")
        torch.save(checkpoint_dict, CHECKPOINTS_DIR / "best.pth")
        best_recall_at_1 = recall_at_n[0]


In [7]:
recall_at_n, recall_at_one_percent, mean_top1_distance = test(
    model=model,
    descriptor_key="fusion",
    dataloader=dataloaders["test"],
    dist_thresh=5.0,
    device=DEVICE,
)

Calculating metrics: 100%|██████████| 6/6 [00:00<00:00,  6.98it/s]

Mean Recall@N:
[0.6942526  0.81604247 0.86556882 0.89520911 0.91542813 0.92709596
 0.93208602 0.9387087  0.94228657 0.94807692 0.95250074 0.9552532
 0.95830611 0.96161686 0.96411157 0.96493305 0.96603619 0.96908523
 0.97047456 0.97242462 0.9743485  0.9760437  0.9774291  0.97825379
 0.97909571]
Mean Recall@1% = 0.9270959598770103
Mean top-1 distance = 0.9369876623933786


## Подготовка ответа для загрузки на сервер

Пример кода для создания файла сабмита. Вы можете модифицировать пайплайн, например добавить ре-ранжирование кандидатов на основе каких-либо характеристик (например, как в [Path-NetVLAD](https://arxiv.org/abs/2103.01486)).

In [12]:
import itertools

import pandas as pd
from sklearn.neighbors import KDTree
import numpy as np
import torch
from tqdm import tqdm


def extract_embeddings(model, descriptor_key, dataloader, device):
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        test_embeddings_list = []
        for data in tqdm(dataloader, desc="Calculating test set descriptors"):
            batch, _, _ = data
            batch = {e: batch[e].to(device) for e in batch}
            batch_embeddings = model(batch)
            test_embeddings_list.append(batch_embeddings[descriptor_key].cpu().numpy())
        test_embeddings = np.vstack(test_embeddings_list)
    return test_embeddings


def test_submission(
    test_embeddings: np.ndarray, dataset_df: pd.DataFrame, filename: str = "submission.txt"
) -> None:
    """Function to create submission txt file.

    Args:
        test_embeddings (np.ndarray): Array of embeddings.
        dataset_df (pd.Dataframe): Test dataset dataframe ('test.csv').
        filename (str): Name of the output txt file. Defaults to "submission.txt".
    """
    tracks = []

    for _, group in dataset_df.groupby("track"):
        tracks.append(group.index.to_numpy())
    n = 1
    ij_permutations = sorted(list(itertools.permutations(range(len(tracks)), 2)))
    # ij_permutations = [(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)]

    submission_lines = []

    for i, j in tqdm(ij_permutations, desc="Calculating metrics"):
        query_indices = tracks[i]
        database_indices = tracks[j]
        query_embs = test_embeddings[query_indices]
        database_embs = test_embeddings[database_indices]

        database_tree = KDTree(database_embs)
        _, indices = database_tree.query(query_embs, k=n)

        submission_lines.extend(list(database_indices[indices.squeeze()]))

    with open(filename, "w") as f:
        for l in submission_lines:
            f.write(str(l)+"\n")


In [13]:
embeddings = extract_embeddings(model, descriptor_key="fusion", dataloader=dataloaders["test"], device=DEVICE)
test_submission(embeddings, dataset_df=dataloaders["test"].dataset.dataset_df, filename="baseline_submission.txt")

Calculating metrics: 100%|██████████| 6/6 [00:00<00:00,  7.67it/s]


Файл с сабмитом необходимо загружать на яндекс контест: https://contest.yandex.ru/contest/49118/ 

Удачи!